In [1]:
import pandas as pd

ct_data = pd.read_csv("CT_2013_2023.csv")
ht_data = pd.read_csv("HT_2013_2023.csv")
leoka_data = pd.read_csv("LEOKA_ASSAULT_TIME_WEAPON_INJURY_1995_2022.csv")

print("CT Data:")
print(ct_data.info())

print("\nHT Data:")
print(ht_data.info())

print("\nLEOKA Data:")
print(leoka_data.info())


CT Data:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 228305 entries, 0 to 228304
Data columns (total 31 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   data_year              228305 non-null  int64  
 1   ori                    228305 non-null  object 
 2   pub_agency_name        228305 non-null  object 
 3   pub_agency_unit        4026 non-null    object 
 4   agency_type_name       228305 non-null  object 
 5   state_abbr             228305 non-null  object 
 6   state_name             228305 non-null  object 
 7   division_name          228305 non-null  object 
 8   county_name            228305 non-null  object 
 9   region_name            228305 non-null  object 
 10  population_group_code  226681 non-null  object 
 11  population_group_desc  226681 non-null  object 
 12  offense_code           228305 non-null  object 
 13  offense_name           228305 non-null  object 
 14  offender_race          1272